In [13]:
#jupyter magic command
%reload_ext autoreload
%autoreload 2

In [11]:
print(os.getcwd())  # Check current working directory


/Users/farshid/taxi_demand_predictor/notebooks


In [12]:
import sys
import os

# Change the current working directory to the root of the project
os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))  # Move up to the root directory

# Add the src directory to the Python path
sys.path.append(os.path.join(os.getcwd(), 'src'))

print(os.getcwd())  # Verify the new working directory
print(sys.path)  # Verify that 'src' is in the Python path

from src.data import load_raw_data

rides = load_raw_data(year=2024)

rides


/Users/farshid/taxi_demand_predictor
['/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python313.zip', '/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13', '/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/lib-dynload', '', '/Users/farshid/Library/Caches/pypoetry/virtualenvs/src-diL_e3P7-py3.13/lib/python3.13/site-packages', '/Users/farshid/taxi_demand_predictor/notebooks/src', '/Users/farshid/taxi_demand_predictor/src']
File 2024-01 was already in local storage
2024-12 file is not available


,pickup_datetime,pickup_location_id
0,2024-01-01 00:57:55,186
1,2024-01-01 00:03:00,140
2,2024-01-01 00:17:06,236
3,2024-01-01 00:36:38,79
4,2024-01-01 00:46:51,211
...,...,...
3646364,2024-11-30 23:11:15,162
3646365,2024-11-30 23:49:30,132
3646366,2024-11-30 23:31:46,100
3646367,2024-11-30 23:41:21,42


In [15]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)
ts_data

/Users/farshid/taxi_demand_predictor/src/data.py:193: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('H')
/Users/farshid/taxi_demand_predictor/src/data.py:156: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_range = pd.date_range(ts_data['pickup_hour'].min(),
100%|██████████| 265/265 [00:01<00:00, 135.33it/s]


,pickup_hour,rides,pickup_location_id
0,2024-01-01 00:00:00,0,1
1,2024-01-01 01:00:00,0,1
2,2024-01-01 02:00:00,0,1
3,2024-01-01 03:00:00,0,1
4,2024-01-01 04:00:00,0,1
...,...,...,...
2130595,2024-11-30 19:00:00,3,265
2130596,2024-11-30 20:00:00,2,265
2130597,2024-11-30 21:00:00,3,265
2130598,2024-11-30 22:00:00,6,265


In [16]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28*1, # one month
    step_size=24,
)

print(f'{features.shape=}')
print(f'{targets.shape=}')

100%|██████████| 265/265 [00:10<00:00, 24.38it/s]

features.shape=(81355, 674)
targets.shape=(81355,)


In [17]:
# save back to disk
tabular_data = features
tabular_data['target_rides_next_hour'] = targets

from src.paths import TRANSFORMED_DATA_DIR
tabular_data.to_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')